### 분석 실습

#### 타이타닉 생존자 예측모델 개발 및 평가(아래 사항들 반영 필수)
- Titanic data 전처리
    - 분석 데이터 : titanic3.csv
    - 재사용 가능한 전처리 사용자 함수 작성 하여 일괄 전처리 Null 값 처리
    - 탐색적 분석을 통한 변수 선정 및 파생변수 생성 
    - 훈련 데이터 : 검증 데이터 = 8:2
    - 분류모델 : Decision Tree, Random Forest, Logistic Regression 3개 모두 적용
    - 평가 : 사이킷런 metrics 사용, 다음 항목들을 포함하는 사용자 함수 def get_clf_eval(y_test, pred)을 작성하여 일괄 처리
    \
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
    - 모델 개발 및 평가 결과에 대한 종합 의견


- 컬럼 정보
    - survived : 생존여부(1: 생존, 0 : 사망)
    - pclass : 승선권 클래스(1 : 1st, 2 : 2nd ,3 : 3rd)
    - name : 승객 이름
    - sex : 승객 성별
    - age : 승객 나이
    - sibsp : 동반한 형제자매, 배우자 수
    - parch : 동반한 부모, 자식 수
    - ticket : 티켓의 고유 넘버
    - fare 티켓의 요금
    - cabin : 객실 번호
    - embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)
    - boat
    - body
    - home.dest


## 1. 탐색적 분석

In [2]:
import seaborn as sns
import pandas as pd

titanic=pd.read_csv('titanic3.csv')
titanic.head(20)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.00,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.00,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.00,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.00,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.00,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [3]:
titanic.info()

#null 값이 존재하는 변수: age, fare, cabin, embarked, boat, body, home.dest

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [4]:
titanic.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [5]:
tit=titanic.copy()
tit.sort_values(['name','ticket'],inplace=True,ignore_index=True)

In [6]:
tit.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3,0,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.5500,NaN,S,NaN,NaN,NaN
1,3,0,"Abbott, Master. Eugene Joseph",male,13.0,0,2,C.A. 2673,20.2500,NaN,S,NaN,NaN,"East Providence, RI"
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.2500,NaN,S,NaN,190.0,"East Providence, RI"
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.2500,NaN,S,A,NaN,"East Providence, RI"
4,3,1,"Abelseth, Miss. Karen Marie",female,16.0,0,0,348125,7.6500,NaN,S,16,NaN,"Norway Los Angeles, CA"
5,3,1,"Abelseth, Mr. Olaus Jorgensen",male,25.0,0,0,348122,7.6500,F G63,S,A,NaN,"Perkins County, SD"
6,2,0,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.0000,NaN,C,NaN,NaN,"Russia New York, NY"
7,2,1,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C,10,NaN,"Russia New York, NY"
8,3,1,"Abrahamsson, Mr. Abraham August Johannes",male,20.0,0,0,SOTON/O2 3101284,7.9250,NaN,S,15,NaN,"Taalintehdas, Finland Hoboken, NJ"
9,3,1,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,C,NaN,"Greensburg, PA"


#### cabin변수 null값 대체: 가족끼리 범주화

In [7]:
# 성만 분리해서 범주화하기

tit['last_name']=tit['name'].apply(lambda x: x.split(',')[0])
tit.head(30)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,last_name
0,3,0,"Abbing, Mr. Anthony",male,42.00,0,0,C.A. 5547,7.5500,NaN,S,NaN,NaN,NaN,Abbing
1,3,0,"Abbott, Master. Eugene Joseph",male,13.00,0,2,C.A. 2673,20.2500,NaN,S,NaN,NaN,"East Providence, RI",Abbott
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.00,1,1,C.A. 2673,20.2500,NaN,S,NaN,190.0,"East Providence, RI",Abbott
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.00,1,1,C.A. 2673,20.2500,NaN,S,A,NaN,"East Providence, RI",Abbott
4,3,1,"Abelseth, Miss. Karen Marie",female,16.00,0,0,348125,7.6500,NaN,S,16,NaN,"Norway Los Angeles, CA",Abelseth
5,3,1,"Abelseth, Mr. Olaus Jorgensen",male,25.00,0,0,348122,7.6500,F G63,S,A,NaN,"Perkins County, SD",Abelseth
6,2,0,"Abelson, Mr. Samuel",male,30.00,1,0,P/PP 3381,24.0000,NaN,C,NaN,NaN,"Russia New York, NY",Abelson
7,2,1,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.00,1,0,P/PP 3381,24.0000,NaN,C,10,NaN,"Russia New York, NY",Abelson
8,3,1,"Abrahamsson, Mr. Abraham August Johannes",male,20.00,0,0,SOTON/O2 3101284,7.9250,NaN,S,15,NaN,"Taalintehdas, Finland Hoboken, NJ",Abrahamsson
9,3,1,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.00,0,0,2657,7.2292,NaN,C,C,NaN,"Greensburg, PA",Abrahim


In [8]:
#중복없는 성의 개수 확인

len(set(tit['last_name']))

875

In [9]:
#성이 같은 사람들은 티켓번호도 다 같은지 확인하기

for lastname in set(tit['last_name']):
    names=tit[tit['last_name']==lastname]
    count=len(set(names['ticket']))
    if count>1:
        idx=names.index
        print(idx)
        print(tit.iloc[list(idx),[2,7,14]])

#다른 데이터도 있음을 확인,ex)idx=31~41

Int64Index([499, 500, 501, 502], dtype='int64')
                                         name    ticket last_name
499         Harper, Miss. Annie Jessie "Nina"    248727    Harper
500                 Harper, Mr. Henry Sleeper  PC 17572    Harper
501  Harper, Mrs. Henry Sleeper (Myna Haxtun)  PC 17572    Harper
502                         Harper, Rev. John    248727    Harper
Int64Index([341, 342, 343], dtype='int64')
                                                  name    ticket last_name
341                         Douglas, Mr. Walter Donald  PC 17761   Douglas
342  Douglas, Mrs. Frederick Charles (Mary Helene B...  PC 17558   Douglas
343        Douglas, Mrs. Walter Donald (Mahala Dutton)  PC 17761   Douglas
Int64Index([149, 150], dtype='int64')
                                              name  ticket last_name
149                   Bradley, Miss. Bridget Delia  334914   Bradley
150  Bradley, Mr. George ("George Arthur Brayton")  111427   Bradley
Int64Index([624, 625, 626], dtype=

Int64Index([883, 884], dtype='int64')
                      name  ticket last_name
883  O'Connor, Mr. Maurice  371060  O'Connor
884  O'Connor, Mr. Patrick  366713  O'Connor
Int64Index([266, 267, 268], dtype='int64')
                  name  ticket last_name
266    Cor, Mr. Bartol  349230       Cor
267      Cor, Mr. Ivan  349229       Cor
268  Cor, Mr. Liudevit  349231       Cor
Int64Index([966, 967, 968], dtype='int64')
                                                  name       ticket last_name
966               Phillips, Miss. Alice Frances Louisa  S.O./P.P. 2  Phillips
967  Phillips, Miss. Kate Florence ("Mrs Kate Louis...       250655  Phillips
968                        Phillips, Mr. Escott Robert  S.O./P.P. 2  Phillips
Int64Index([465, 466, 467], dtype='int64')
                                              name    ticket last_name
465                   Graham, Miss. Margaret Edith    112053    Graham
466                      Graham, Mr. George Edward  PC 17582    Graham
467  Grah

Int64Index([395, 396, 397, 398, 399, 400], dtype='int64')
                                        name      ticket last_name
395     Ford, Miss. Doolina Margaret "Daisy"  W./C. 6608      Ford
396         Ford, Miss. Robina Maggie "Ruby"  W./C. 6608      Ford
397                         Ford, Mr. Arthur    A/5 1478      Ford
398                  Ford, Mr. Edward Watson  W./C. 6608      Ford
399                   Ford, Mr. William Neal  W./C. 6608      Ford
400  Ford, Mrs. Edward (Margaret Ann Watson)  W./C. 6608      Ford
Int64Index([1016, 1017], dtype='int64')
                            name           ticket last_name
1016  Rogers, Mr. Reginald Harry            28004    Rogers
1017    Rogers, Mr. William John  S.C./A.4. 23567    Rogers
Int64Index([1284, 1285], dtype='int64')
                      name  ticket last_name
1284  Wright, Miss. Marion  220844    Wright
1285    Wright, Mr. George  113807    Wright
Int64Index([366, 367, 368, 369], dtype='int64')
                     name tick

In [10]:
# 성이 같고, 티켓 번호가 같은 사람들끼리 그룹화

for lastname in set(tit['last_name']):
    names=tit[tit['last_name']==lastname]
    named_ticket=set(names['ticket'])
    count=len(named_ticket)
    if count>1:
        category=0
        for tic in named_ticket:
            tit.loc[(tit['last_name']==lastname)&(tit['ticket']==tic),'last_name']=tit['last_name']+'_'+str(category)
            category+=1

In [11]:
#그룹화가 잘되어있는지 확인
#성이 같은 사람들은 티켓번호도 다 같은지 확인하기

for lastname in set(tit['last_name']):
    names=tit[tit['last_name']==lastname]
    count=len(set(names['ticket']))
    if count>1:
        idx=names.index
        print(idx)
        print(tit.iloc[list(idx),[2,7,14]])   #full name, ticket 번호, lastname 출력

In [12]:
#중복없는 성의 개수 확인

len(set(tit.last_name))

1024

In [13]:
#데이터로 확인

tit.iloc[31:41]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,last_name
31,3,0,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
32,3,0,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
33,3,0,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
34,3,1,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.9250,NaN,S,D,NaN,"Ruotsinphyhtaa, Finland New York, NY",Andersson_4
35,3,0,"Andersson, Miss. Ida Augusta Margareta",female,38.0,4,2,347091,7.7750,NaN,S,NaN,NaN,"Vadsbro, Sweden Ministee, MI",Andersson_1
36,3,0,"Andersson, Miss. Ingeborg Constanzia",female,9.0,4,2,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
37,3,0,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
38,3,0,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,NaN,NaN,"Sweden Winnipeg, MN",Andersson_2
39,3,1,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S,A,NaN,NaN,Andersson_0
40,3,0,"Andersson, Mr. Johan Samuel",male,26.0,0,0,347075,7.7750,NaN,S,NaN,NaN,"Hartford, CT",Andersson_3


이 과정을 진행한 이유는 결측값이 많은 cabin을 대체하기 위해서임.
cabin은 선실번호로, 성이 같은 가족은 cabin도 같을 것으로 추정됨
따라서, last_name 변수의 이름을 나중에 인코딩할 때 활용하도록 cobin_category로 바꾸겠음

In [14]:
tit2=tit.rename(columns={'last_name':'cabin_category'})
tit2.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cabin_category
0,3,0,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S,NaN,NaN,NaN,Abbing
1,3,0,"Abbott, Master. Eugene Joseph",male,13.0,0,2,C.A. 2673,20.25,NaN,S,NaN,NaN,"East Providence, RI",Abbott
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S,NaN,190.0,"East Providence, RI",Abbott
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S,A,NaN,"East Providence, RI",Abbott
4,3,1,"Abelseth, Miss. Karen Marie",female,16.0,0,0,348125,7.65,NaN,S,16,NaN,"Norway Los Angeles, CA",Abelseth_0


#### age변수 null값 대체

In [15]:
tit3=tit2.copy()
tit3.corr()

#pclass와 상관성이 상대적으로 높아보임

,pclass,survived,age,sibsp,parch,fare,body
pclass,1.000000,-0.312469,-0.408106,0.060832,0.018322,-0.558629,-0.034642
survived,-0.312469,1.000000,-0.055512,-0.027825,0.082660,0.244265,NaN
age,-0.408106,-0.055512,1.000000,-0.243699,-0.150917,0.178740,0.058809
sibsp,0.060832,-0.027825,-0.243699,1.000000,0.373587,0.160238,-0.099961
parch,0.018322,0.082660,-0.150917,0.373587,1.000000,0.221539,0.051099
fare,-0.558629,0.244265,0.178740,0.160238,0.221539,1.000000,-0.043110
body,-0.034642,NaN,0.058809,-0.099961,0.051099,-0.043110,1.000000


In [16]:
#생존여부별 좌석 등급별 나이 평균값, 최소값, 최대값

grouped_p=tit3.groupby('pclass')
grouped_p.agg({'age':['mean','min','max','count']})

#(max-min)범위가 너무 커서 평균값 대체가 의미가 없어보임

age                  
             mean   min   max count
pclass                             
1       39.159930  0.92  80.0   284
2       29.506705  0.67  70.0   261
3       24.816367  0.17  74.0   501

In [17]:
grouped_p.age.describe().T

#평균값과 중앙값이 차이가 없어보임

pclass,1,2,3
count,284.000000,261.000000,501.000000
mean,39.159930,29.506705,24.816367
std,14.548028,13.638627,11.958202
min,0.920000,0.670000,0.170000
25%,28.000000,22.000000,18.000000
50%,39.000000,29.000000,24.000000
75%,50.000000,36.000000,32.000000
max,80.000000,70.000000,74.000000


In [18]:
#생존여부별/좌석 등급별 나이 평균값, 최소값, 최대값
grouped_p=tit2.groupby(['survived','pclass'])
grouped_p.agg({'age':['mean','min','max','count']})

#(max-min)범위가 너무 커서 평균값 대체가 의미가 없어보임

age                   
                      mean    min   max count
survived pclass                              
0        1       43.247573   2.00  71.0   103
         2       33.167808  14.00  70.0   146
         3       25.974541   0.33  74.0   370
1        1       36.833812   0.92  80.0   181
         2       24.858696   0.67  62.0   115
         3       21.545191   0.17  63.0   131

In [19]:
#세부 그룹화
#생존여부, 좌석등급 및 형제자매수, 부모자식수에 따라서도 그룹화

grouped_pf=tit3.groupby(['survived','pclass','sibsp','parch'])
grouped_pf=grouped_pf.agg({'age':['mean','min','max','count']}).round(2)
grouped_pf

age                   
                              mean    min   max count
survived pclass sibsp parch                          
0        1      0     0      43.96  17.00  71.0    58
                      1      41.43  21.00  65.0     7
                      2      42.50  27.00  58.0     2
                1     0      41.59  18.00  67.0    22
                      1      52.86  37.00  70.0     7
...                            ...    ...   ...   ...
1        3      2     0        NaN    NaN   NaN     0
                      1       2.17   0.75   5.0     3
                3     0      33.00  33.00  33.0     1
                      1      29.00  29.00  29.0     1
                4     2       8.33   3.00  17.0     3

[75 rows x 4 columns]

In [20]:
#대체해야할 survived, pclass, sibsp, parch의 중복값 제거한 행 살펴보기

age_null=tit3[tit3['age'].isnull()]
age_null_s=age_null[['survived','pclass','sibsp','parch']].sort_values(['survived','pclass','sibsp','parch'])
age_null_sd=age_null_s.drop_duplicates()
age_null_sd

,survived,pclass,sibsp,parch
97,0,1,0,0
190,0,2,0,0
118,0,3,0,0
140,0,3,0,2
693,0,3,0,4
195,0,3,1,0
1305,0,3,1,1
607,0,3,1,2
1058,0,3,1,9
1065,0,3,2,0


In [21]:
s.columns

NameError: name 's' is not defined

In [ ]:
s=grouped_pf.reset_index()
s

In [22]:
# age null 값을 생존여부, 좌석등급, 형제자매수, 부모자식수 범주 구간별 평균값으로 대체하기
import numpy as np

idx=list(tit3[tit3['age'].isnull()].index)


In [23]:
tit3.isnull().sum()

pclass               0
survived             0
name                 0
sex                  0
age                263
sibsp                0
parch                0
ticket               0
fare                 1
cabin             1014
embarked             2
boat               823
body              1188
home.dest          564
cabin_category       0
dtype: int64

In [24]:
#age null값 대체하기
#좌석등급, 가족 수에 따라 null값을 평균값으로 대체
tit3=tit2.copy()
li=[[1,0,0,40.49],[1,1,0,38.73],[1,0,1,36.77],[2,0,0,32.22],[3,0,0,27.61],
      [3,0,2,22.39],[3,1,1,16.75],[3,1,0,26.45],[3,8,2,14.50],
      [3,1,9,41.50],[3,2,0,25.00],[3,1,2,15.04],[3,3,1,9.60],[3,0,4,29.00]]

for i,j,k,m in li:
    tit3.loc[(tit3['pclass']==i)&(tit3['sibsp']==j)&(tit3['parch']==k),'age']=m

tit3.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cabin_category
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,NaN,NaN,NaN,Abbing
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,NaN,NaN,"East Providence, RI",Abbott
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,NaN,190.0,"East Providence, RI",Abbott
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,A,NaN,"East Providence, RI",Abbott
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,16,NaN,"Norway Los Angeles, CA",Abelseth_0


In [25]:
#null값 여부 확인
tit3.isnull().sum()

pclass               0
survived             0
name                 0
sex                  0
age                  0
sibsp                0
parch                0
ticket               0
fare                 1
cabin             1014
embarked             2
boat               823
body              1188
home.dest          564
cabin_category       0
dtype: int64

In [26]:
#age 범주화
def age_category(x):
    y=0
    if x<10:
        y='baby'
    elif x<20:
        y='10대'
    elif x<30:
        y='20대'
    elif x<40:
        y='30대'
    elif x<50:
        y='40대'
    elif x<60:
        y='50대'
    elif x>=60:
        y='60대'
    return y

tit3['age_c']=tit3['age'].apply(age_category)
tit3.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cabin_category,age_c
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,NaN,NaN,NaN,Abbing,20대
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,NaN,NaN,"East Providence, RI",Abbott,20대
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,NaN,190.0,"East Providence, RI",Abbott,10대
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,A,NaN,"East Providence, RI",Abbott,10대
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,16,NaN,"Norway Los Angeles, CA",Abelseth_0,20대


In [27]:
tit3.age_c.value_counts()

20대     626
30대     305
40대     195
10대      90
baby     66
50대      17
60대      10
Name: age_c, dtype: int64

#### body변수:  null값 삭제

In [28]:
tit4=tit3.copy()
tit4.isnull().sum()

#null값이 너무 많음

pclass               0
survived             0
name                 0
sex                  0
age                  0
sibsp                0
parch                0
ticket               0
fare                 1
cabin             1014
embarked             2
boat               823
body              1188
home.dest          564
cabin_category       0
age_c                0
dtype: int64

In [29]:
tit4.corr()

#연령 변수와 그나마 상관성이 있어보임

,pclass,survived,age,sibsp,parch,fare,body
pclass,1.000000,-0.312469,-0.592451,0.060832,0.018322,-0.558629,-0.034642
survived,-0.312469,1.000000,0.056325,-0.027825,0.082660,0.244265,NaN
age,-0.592451,0.056325,1.000000,-0.349269,-0.195590,0.277276,0.131111
sibsp,0.060832,-0.027825,-0.349269,1.000000,0.373587,0.160238,-0.099961
parch,0.018322,0.082660,-0.195590,0.373587,1.000000,0.221539,0.051099
fare,-0.558629,0.244265,0.277276,0.160238,0.221539,1.000000,-0.043110
body,-0.034642,NaN,0.131111,-0.099961,0.051099,-0.043110,1.000000


In [30]:
#대체값을 찾기 위해 연령을 범주화해보겠음

import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
tit4['age_c_l']=le.fit_transform(tit4[['age_c']])
tit4.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cabin_category,age_c,age_c_l
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,NaN,NaN,NaN,Abbing,20대,1
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,NaN,NaN,"East Providence, RI",Abbott,20대,1
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,NaN,190.0,"East Providence, RI",Abbott,10대,0
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,A,NaN,"East Providence, RI",Abbott,10대,0
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,16,NaN,"Norway Los Angeles, CA",Abelseth_0,20대,1


In [31]:
tit4.corr()

#오히려 낮아진것을 확인할 수 있음

,pclass,survived,age,sibsp,parch,fare,body,age_c_l
pclass,1.000000,-0.312469,-0.592451,0.060832,0.018322,-0.558629,-0.034642,-0.459438
survived,-0.312469,1.000000,0.056325,-0.027825,0.082660,0.244265,NaN,0.157906
age,-0.592451,0.056325,1.000000,-0.349269,-0.195590,0.277276,0.131111,0.093434
sibsp,0.060832,-0.027825,-0.349269,1.000000,0.373587,0.160238,-0.099961,0.134165
parch,0.018322,0.082660,-0.195590,0.373587,1.000000,0.221539,0.051099,0.191738
fare,-0.558629,0.244265,0.277276,0.160238,0.221539,1.000000,-0.043110,0.236282
body,-0.034642,NaN,0.131111,-0.099961,0.051099,-0.043110,1.000000,0.030626
age_c_l,-0.459438,0.157906,0.093434,0.134165,0.191738,0.236282,0.030626,1.000000


In [32]:
#삭제
tit5=tit4.drop(['body','age_c_l'],axis=1)
tit5.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest,cabin_category,age_c
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,NaN,NaN,Abbing,20대
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,NaN,"East Providence, RI",Abbott,20대
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,NaN,"East Providence, RI",Abbott,10대
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,A,"East Providence, RI",Abbott,10대
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,16,"Norway Los Angeles, CA",Abelseth_0,20대


#### boat 변수:  null값 삭제

In [33]:
tit5.isnull().sum()

#null값이 너무 많음

pclass               0
survived             0
name                 0
sex                  0
age                  0
sibsp                0
parch                0
ticket               0
fare                 1
cabin             1014
embarked             2
boat               823
home.dest          564
cabin_category       0
age_c                0
dtype: int64

In [34]:
tit5.boat.unique()

array([nan, 'A', '16', '10', '15', 'C', '11', '13', '2', '3', 'D', '4',
       '9', '6', 'B', '8', '5', '12', '7', '14', '13 15 B', '5 9', '1',
       'C D', '15 16', '5 7', '13 15', '8 10'], dtype=object)

In [35]:
tit5[tit5['boat']=='15 16']

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest,cabin_category,age_c
768,3,1,"McCarthy, Miss. Catherine ""Katie""",female,27.61,0,0,383123,7.75,NaN,Q,15 16,NaN,McCarthy_1,20대


In [36]:
tit5[tit5['boat']=='5']

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest,cabin_category,age_c
110,1,1,"Beckwith, Mr. Richard Leonard",male,37.00,1,1,11751,52.5542,D35,S,5,"New York, NY",Beckwith,30대
111,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.00,1,1,11751,52.5542,D35,S,5,"New York, NY",Beckwith,40대
113,1,1,"Behr, Mr. Karl Howell",male,40.49,0,0,111369,30.0000,C148,C,5,"New York, NY",Behr,40대
183,1,1,"Calderhead, Mr. Edward Pennington",male,40.49,0,0,PC 17476,26.2875,E24,S,5,"New York, NY",Calderhead,40대
215,1,1,"Cassebeer, Mrs. Henry Arthur Jr (Eleanor Genev...",female,40.49,0,0,17770,27.7208,NaN,C,5,"New York, NY",Cassebeer,40대
221,1,1,"Chambers, Mr. Norman Campbell",male,38.73,1,0,113806,53.1000,E8,S,5,"New York, NY / Ithaca, NY",Chambers,30대
222,1,1,"Chambers, Mrs. Norman Campbell (Bertha Griggs)",female,38.73,1,0,113806,53.1000,E8,S,5,"New York, NY / Ithaca, NY",Chambers,30대
334,1,1,"Dodge, Master. Washington",male,4.00,0,2,33638,81.8583,A34,S,5,"San Francisco, CA",Dodge,baby
335,1,1,"Dodge, Mrs. Washington (Ruth Vidaver)",female,54.00,1,1,33638,81.8583,A34,S,5,"San Francisco, CA",Dodge,50대
391,1,1,"Flynn, Mr. John Irwin (""Irving"")",male,40.49,0,0,PC 17474,26.3875,E25,S,5,"Brooklyn, NY",Flynn_0,40대


In [37]:
tit5[tit5['boat']=='16']

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest,cabin_category,age_c
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.6500,NaN,S,16,"Norway Los Angeles, CA",Abelseth_0,20대
29,3,1,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,26.45,1,0,350046,7.8542,NaN,S,16,NaN,Andersen-Jensen,20대
203,3,1,"Carr, Miss. Helen ""Ellen""",female,27.61,0,0,367231,7.7500,NaN,Q,16,"Co Longford, Ireland New York, NY",Carr_1,20대
314,3,1,"Davison, Mrs. Thomas Henry (Mary E Finck)",female,26.45,1,0,386525,16.1000,NaN,S,16,"Liverpool, England Bedford, OH",Davison,20대
360,3,1,"Dyker, Mrs. Adolf Fredrik (Anna Elisabeth Judi...",female,26.45,1,0,347072,13.9000,NaN,S,16,"West Haven, CT",Dyker,20대
382,2,1,"Faunthorpe, Mrs. Lizzie (Elizabeth Anne Wilkin...",female,29.00,1,0,2926,26.0000,NaN,S,16,NaN,Faunthorpe,20대
445,3,1,"Gilnagh, Miss. Katherine ""Katie""",female,27.61,0,0,35851,7.7333,NaN,Q,16,"Co Longford, Ireland New York, NY",Gilnagh,20대
520,3,1,"Healy, Miss. Hanora ""Nora""",female,27.61,0,0,370375,7.7500,NaN,Q,16,NaN,Healy,20대
637,3,1,"Kelly, Miss. Anna Katherine ""Annie Kate""",female,27.61,0,0,9234,7.7500,NaN,Q,16,NaN,Kelly_1,20대
755,3,1,"Mannion, Miss. Margareth",female,27.61,0,0,36866,7.7375,NaN,Q,16,NaN,Mannion,20대


In [38]:
#규칙성이 보이지 않아 삭제

tit6=tit5.drop('boat',axis=1)
tit6.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest,cabin_category,age_c
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,NaN,Abbing,20대
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,"East Providence, RI",Abbott,20대
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI",Abbott,10대
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI",Abbott,10대
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,"Norway Los Angeles, CA",Abelseth_0,20대


#### home.dest 변수 삭제

In [39]:
tit7=tit6.drop('home.dest',axis=1)
tit7.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,cabin_category,age_c
0,3,0,"Abbing, Mr. Anthony",male,27.61,0,0,C.A. 5547,7.55,NaN,S,Abbing,20대
1,3,0,"Abbott, Master. Eugene Joseph",male,22.39,0,2,C.A. 2673,20.25,NaN,S,Abbott,20대
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.75,1,1,C.A. 2673,20.25,NaN,S,Abbott,10대
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,16.75,1,1,C.A. 2673,20.25,NaN,S,Abbott,10대
4,3,1,"Abelseth, Miss. Karen Marie",female,27.61,0,0,348125,7.65,NaN,S,Abelseth_0,20대


#### fare 변수 null값 대체

In [40]:
tit7.isnull().sum()

pclass               0
survived             0
name                 0
sex                  0
age                  0
sibsp                0
parch                0
ticket               0
fare                 1
cabin             1014
embarked             2
cabin_category       0
age_c                0
dtype: int64

In [41]:
tit7.corr()

#pclass, survived 변수들과 상관성이 어느정도 나타남

,pclass,survived,age,sibsp,parch,fare
pclass,1.000000,-0.312469,-0.592451,0.060832,0.018322,-0.558629
survived,-0.312469,1.000000,0.056325,-0.027825,0.082660,0.244265
age,-0.592451,0.056325,1.000000,-0.349269,-0.195590,0.277276
sibsp,0.060832,-0.027825,-0.349269,1.000000,0.373587,0.160238
parch,0.018322,0.082660,-0.195590,0.373587,1.000000,0.221539
fare,-0.558629,0.244265,0.277276,0.160238,0.221539,1.000000


In [42]:
#피벗테이블을 이용해 좌석등급별, 생존여부별 fare의 평균값, 최소값, 최댓값 구하기

fare_p=pd.pivot_table(tit7, index='pclass', columns='survived',values='fare',aggfunc=['mean','min','max','count'])
fare_p

#특이하게도, 좌석등급이 높아도 최소값이 0임.

mean             min           max           count     
survived          0          1    0     1       0         1     0    1
pclass                                                                
1         70.412533  98.023314  0.0   0.0  263.00  512.3292   123  200
2         19.801713  23.008124  0.0  10.5   73.50   65.0000   158  119
3         13.435451  12.916919  0.0   0.0   69.55   56.4958   527  181

In [43]:
#데이터 살펴보기

tit7[tit7['fare']<7]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,cabin_category,age_c
46,1,0,"Andrews, Mr. Thomas Jr",male,40.49,0,0,112050,0.0000,A36,S,Andrews_0,40대
119,3,0,"Betros, Mr. Tannous",male,27.61,0,0,2648,4.0125,NaN,C,Betros_0,20대
170,3,0,"Burke, Mr. Jeremiah",male,27.61,0,0,365222,6.7500,NaN,Q,Burke,20대
190,2,0,"Campbell, Mr. William",male,32.22,0,0,239853,0.0000,NaN,S,Campbell,30대
202,1,0,"Carlsson, Mr. Frans Olof",male,40.49,0,0,695,5.0000,B51 B53 B55,S,Carlsson_1,40대
232,1,0,"Chisholm, Mr. Roderick Robert Crispin",male,40.49,0,0,112051,0.0000,NaN,S,Chisholm,40대
288,2,0,"Cunningham, Mr. Alfred Fleming",male,32.22,0,0,239853,0.0000,NaN,S,Cunningham,30대
293,3,1,"Daly, Miss. Margaret Marcella ""Maggie""",female,27.61,0,0,382650,6.9500,NaN,Q,Daly_0,20대
365,3,0,"Ekstrom, Mr. Johan",male,27.61,0,0,347061,6.9750,NaN,S,Ekstrom,20대
390,3,0,"Flynn, Mr. John",male,27.61,0,0,368323,6.9500,NaN,Q,Flynn_2,20대


In [44]:
#등급별,생존여부 boxplot 그려보기
grouped_ps=tit7.groupby(['pclass','survived'])
grouped_ps

In [45]:


sns.boxplot(,'fare',data=tit7)

SyntaxError: invalid syntax (1381600770.py, line 1)

In [ ]:
#범주화 후 labelencoding

from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()


In [ ]:
tit_f.loc[tit_f['fare'].isnull()]

In [ ]:
tit_f.loc[tit_f['pclass']==3,'fare'].mean()

In [ ]:
tit_f=tit7.copy()

tit_f.sex=tit_f.sex.replace(['male','female'],[0,1])
tit_f['cabin_c']=le.fit_transform(tit_f[['cabin_category']])
tit_f['embarked']=le.fit_transform(tit_f[['embarked']])
tit_f['embarked']=tit_f['embarked'].fillna(method='ffill')
tit_f.loc[tit_f['fare'].isnull(),'fare']=tit_f.loc[tit_f['pclass']==3,'fare'].mean()

tit_f.head()
tit_f.isnull().sum()

In [ ]:
tit_f=tit7.copy()

tit_f.sex=tit_f.sex.replace(['male','female'],[0,1])
tit_f.loc[tit_f['fare'].isnull(),'fare']=tit_f.loc[tit_f['pclass']==3,'fare'].mean()


tit_f['embarked']=tit_f['embarked'].fillna(method='ffill')
tit_f=pd.get_dummies(tit_f[['cabin_category','embarked']])



tit_f.head()
tit_f.isnull().sum()

## 2. 분석용 데이터셋 준비

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


#독립변수, 종속변수 분리(변수는 바꿔도 됨!)
X = tit_f[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin_c', 'embarked']]
y = tit_f['survived']

# 학습용 테이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, \
                                                    random_state=10) 
print(X_train.shape)
print(X_test.shape)


## 3. 모델링 및 평가

In [ ]:
# Decision Tree
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

print(dt_pred[0:10])
print(y_test.values[0:10])

accuracy = accuracy_score(y_test, dt_pred)
print('dt 예측 정확도 :', accuracy)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_pred)
print('rf 예측 정확도 :', rf_accuracy)

In [ ]:
# Logistic Regression
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=0)
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)

lr_accuracy = accuracy_score(y_test, lr_pred)
print('lr 예측 정확도 :', accuracy)
